In [ ]:
import rospy
import numpy as np
from panda_robot_custom import PandaRobotCustom
from tf.transformations import quaternion_from_euler, euler_from_quaternion
import my_log
import tools

log = my_log.logger()
log._print_logger_info()

rospy.init_node("franka_test", anonymous=True)

robot = PandaRobotCustom(log_level=10)
rospy.sleep(1) 

In [ ]:
trans = [0.3, 0.0, 0.6]
rot = [np.pi, 0.0, 0.0]
robot.cart_move(trans, quaternion_from_euler(rot[0], rot[1], rot[2]))

In [1]:
log.info(robot.get_cart_pose())

NameError: name 'log' is not defined